In [1]:
import torch, os
from diffusers import StableDiffusionImg2ImgPipeline

MODEL_DIR = "/kaggle/input/styleid_stable_diffusion/transformers/default/1/stable-diffusion-v1-5"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_DIR,
    torch_dtype=torch.float16,
    safety_checker=None,
    feature_extractor=None
).to("cuda")

# 节省显存的一些小优化
pipe.enable_attention_slicing()
pipe.enable_vae_tiling()


2025-08-24 12:28:00.988227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756038481.183781      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756038481.245409      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [2]:
import glob, os, math, hashlib
from PIL import Image
from tqdm import tqdm
import torch

IN_DIR  = "/kaggle/input/nyu-rgb"
OUT_DIR = "/kaggle/working/stylized"  # 输出目录
os.makedirs(OUT_DIR, exist_ok=True)

# 文本风格提示

PROMPT = "impressionist painting, broken color, visible brushwork, warm palette"
NEG = "deformed, distorted, warped, extra objects, changed layout, low quality"

NUM_STEPS       = 25
GUIDANCE_SCALE  = 5
STRENGTH        = 0.35
SEED_BASE       = 42

def to_multiple_of_8(w, h, max_side=768):
    # Diffusers 支持任意尺寸，尽量对齐到 8 的倍数，并限制最长边避免显存爆
    scale = 1.0
    if max(w, h) > max_side:
        scale = max_side / max(w, h)
    w2 = int(math.floor((w * scale) / 8) * 8)
    h2 = int(math.floor((h * scale) / 8) * 8)
    return max(64, w2), max(64, h2)

def seed_from_name(name, base=SEED_BASE):
    h = int(hashlib.sha1(name.encode("utf-8")).hexdigest(), 16) % (2**31)
    return (h ^ base) % (2**31)

paths = sorted(glob.glob(os.path.join(IN_DIR, "*.png")) + 
               glob.glob(os.path.join(IN_DIR, "*.jpg")) + 
               glob.glob(os.path.join(IN_DIR, "*.jpeg")))

print(f"Found {len(paths)} images in {IN_DIR}")

for p in tqdm(paths):
    fname = os.path.basename(p)
    stem, ext = os.path.splitext(fname)
    out_path = os.path.join(OUT_DIR, f"{stem}.png")
    if os.path.exists(out_path):
        continue

    # 读图并尺寸规整
    img = Image.open(p).convert("RGB")
    w, h = img.size
    w2, h2 = to_multiple_of_8(w, h, max_side=768)  # 若显存足够可调大到 1024
    if (w2, h2) != (w, h):
        img = img.resize((w2, h2), Image.LANCZOS)

    # 每张图用文件名派生 seed，保证可复现
    g = torch.Generator(device="cuda").manual_seed(seed_from_name(stem))

    with torch.autocast("cuda"):
        out = pipe(
            prompt=PROMPT,
            negative_prompt=NEG,
            image=img,
            strength=STRENGTH,
            guidance_scale=GUIDANCE_SCALE,
            num_inference_steps=NUM_STEPS,
            generator=g
        ).images[0]

    out.save(out_path)

print("Done. Styled images ->", OUT_DIR)

# 打包 ZIP 方便下载
import shutil
zip_path = "/kaggle/working/stylized_outputs"
shutil.make_archive(zip_path, "zip", OUT_DIR)
print("Zipped to:", zip_path + ".zip")


Found 666 images in /kaggle/input/nyu-rgb


  0%|          | 0/666 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 1/666 [00:07<1:19:29,  7.17s/it]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

100%|█████████▉| 665/666 [1:08:50<00:06,  6.22s/it]

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 666/666 [1:08:56<00:00,  6.21s/it]


Done. Styled images -> /kaggle/working/stylized
Zipped to: /kaggle/working/stylized_outputs.zip
